In [1]:
#Author-Vishal Burman

## Concise Implementation of Linear Regression

In [2]:
from mxnet import autograd, nd, gluon

In [3]:
#Generating datasets as per the last notebook

In [4]:
true_w=nd.array([2, -3.4])
true_b=4.2

In [5]:
def synthetic_data(w, b, num_examples):
    X=nd.random.normal(scale=1, shape=(num_examples, len(w)))
    y=nd.dot(X, w)+b
    y+=nd.random.normal(scale=0.01, shape=y.shape)
    return X, y

In [6]:
features, labels=synthetic_data(true_w, true_b, 1000)

## Reading data

In [ ]:
# We use Gluons's data module to read the data
# shuffle-shuffles the dataset through each epoch

In [7]:
def load_array(data_arrays, batch_size, is_train=True):
    """Construct a Gluon data loader"""
    dataset=gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
batch_size=10
data_iter=load_array((features, labels), batch_size)

In [8]:
# We can go through the data_iter

In [9]:
for X, y in data_iter:
    print(X, y)
    break


[[ 0.29402137 -1.5258902 ]
 [ 1.3810002  -0.76296663]
 [ 1.3894764  -0.18249638]
 [-0.950482   -0.373658  ]
 [ 0.70023924  0.6507114 ]
 [-0.4631116  -0.53839856]
 [ 0.02718297  1.1702743 ]
 [ 1.999043    0.15202568]
 [-1.9525608   0.8395779 ]
 [-0.4847113   1.1821395 ]]
<NDArray 10x2 @cpu(0)> 
[ 9.98281    9.539722   7.6058087  3.5709095  3.397948   5.1078997
  0.2818811  7.7017674 -2.5533025 -0.7929086]
<NDArray 10 @cpu(0)>


## Define the Model

In [ ]:
# There is a fully connected layer in Linear Regression
# In Gluon the fully connected layer is defined by Dense class
# We want to generate a single output therefore the (1) in Dense class

In [10]:
from mxnet.gluon import nn

In [11]:
net=nn.Sequential()

In [12]:
net.add(nn.Dense(1))

## Initialize Model Parameters

In [ ]:
# We need to initialize the weights and biases  in the linear regression model
# Gluon provides the init to access the initializer package
# The weights will be sampled from Gaussian distribution with mean 0 and std. deviation by 0.01
# The bias

In [13]:
from mxnet import init

In [14]:
net.initialize(init.Normal(sigma=0.01))

## Define the Loss Function

In [15]:
from mxnet.gluon import loss as gloss

In [16]:
loss=gloss.L2Loss()

## Define the Optimization Algorithm

In [17]:
trainer=gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.03})

## Training

In [18]:
num_epochs=3
for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        with autograd.record():
            l=loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l=loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean().asnumpy()))

epoch 1, loss: 0.035068
epoch 2, loss: 0.000130
epoch 3, loss: 0.000049


In [19]:
net[0].weight.data()


[[ 1.9998931 -3.3992453]]
<NDArray 1x2 @cpu(0)>

In [20]:
net[0].bias.data()


[4.200356]
<NDArray 1 @cpu(0)>